In [3]:
import torch
from torch import nn
from torch.quantization import get_default_qconfig, prepare, convert
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Subset, random_split
from Model.Schema import resModel
from tqdm import tqdm
from PIL import Image


model_path = "Weights/model_13.pth"
num_classes = 70
model = resModel(num_classes)
model.load_state_dict(torch.load(model_path))
model.eval()


resModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [4]:
supported_layers = {nn.Linear, nn.Conv2d, nn.LSTM, nn.GRU}

model_quantized = torch.quantization.quantize_dynamic(
    model, supported_layers, dtype=torch.qint8
)

In [5]:
model_quantized_path = "Weights/model_13_quantized_dynamics.pth"
torch.save(model_quantized.state_dict(), model_quantized_path)


In [6]:
test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

image_path = 'dataset/valid/American  Spaniel/07.jpg'
image = Image.open(image_path).convert('RGB')

image = test_transform(image).unsqueeze(0)


model_quantized.eval()

device = torch.device('cpu')
model_quantized.to(device)
image = image.to(device)

with torch.no_grad():
    output = model_quantized(image)
    prediction = torch.argmax(output, dim=1)

print(prediction.item())

43


In [8]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from Model.Schema import resModel
import wandb
from tqdm import tqdm


test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
test_dataset = datasets.ImageFolder(root='dataset/test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def test_model(model, test_loader, device):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc="Testing", leave=True):
            data, target = data.to(device), target.to(device)

            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    test_accuracy = 100 * correct / total
    return test_accuracy

device= torch.device('cpu')
model_quantized.to(device)

# accuracy_quantized_dynamics = test_model(model_quantized, test_loader, device)
# wandb.log({"Test Accuracy": accuracy})
# print(f"Test Accuracy of the quantized model: {accuracy_quantized_dynamics}%")

# accuracy_quantized_static = test_model(model_quantized_static, test_loader, device)
# # wandb.log({"Test Accuracy": accuracy})
# print(f"Test Accuracy of the static quantized model: {accuracy_quantized_static}%")

# accuracy_standard = test_model(model, test_loader, device)
# wandb.log({"Test Accuracy": accuracy})
# print(f"Test Accuracy of the standard model: {accuracy_standard}%")




resModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [9]:
import time

start_time_q = time.time()
accuracy_q = test_model(model_quantized, test_loader, device)
end_time_q = time.time()


print(f"Test Accuracy of the dynamics quantized model: {accuracy_q}%")
print(f"Time taken to test the dynamics quantized model: {end_time_q - start_time_q} seconds")



start_time_s = time.time()
accuracy_s = test_model(model, test_loader, device)
end_time_s = time.time()

print(f"Test Accuracy of the standard model: {accuracy_s}%")
print(f"Time taken to test the standard model: {end_time_s - start_time_s} seconds")

Testing: 100%|█████████████████████████████████████████████████████████████████████████| 11/11 [00:17<00:00,  1.60s/it]


Test Accuracy of the dynamics quantized model: 86.28571428571429%
Time taken to test the dynamics quantized model: 17.65607190132141 seconds


Testing: 100%|█████████████████████████████████████████████████████████████████████████| 11/11 [00:17<00:00,  1.55s/it]

Test Accuracy of the standard model: 86.42857142857143%
Time taken to test the standard model: 17.027645111083984 seconds
